In [ ]:
!git clone https://github.com/CallumRai/Hansard/
!pip install transformers

Cloning into 'Hansard'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 113 (delta 42), reused 68 (delta 28), pack-reused 30
Receiving objects: 100% (113/113), 6.00 MiB | 16.18 MiB/s, done.
Resolving deltas: 100% (49/49), done.
     |████████████████████████████████| 1.9MB 5.3MB/s 
     |████████████████████████████████| 3.2MB 36.3MB/s 
     |████████████████████████████████| 890kB 50.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=e626a91d92bd8bdbe6d8d02298d9fdc1a4493e33aef421aeb50b4fc8858fa487
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import os
os.chdir("Hansard")

from hansard import Hansard, Corpus, DataLoader, Trainer
import torch

In [ ]:
print("Downloading and extracting hansard ...")

hansard = Hansard("2019-01-01", "2020-06-01")
hansard.download()
hansard.extract()

In [ ]:
print("Creating corpus ...")

corpus = Corpus("2019-01-01", "2020-06-01")
corpus.full()

Creating corpus ...


In [ ]:
print("Creating dataloaders ...")

dataloader_path = "hansard/data/dataloaders/"
train_path = dataloader_path + 'train_loader_debug.pth'
val_path = dataloader_path + 'val_loader_debug.pth'

# If train or val loaders files do not exist make them
if not(os.path.isfile(train_path) or os.path.isfile(val_path)):

    loader_class = DataLoader("2019-01-01", "2020-06-01")
    train_loader, val_loader = loader_class.debug(0.75, 1)

    torch.save(train_loader, train_path)
    torch.save(val_loader, val_path)

train_loader = torch.load(train_path)
val_loader = torch.load(val_path)

Creating dataloaders ...


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
trainer = Trainer(train_loader, val_loader)

epochs = 5
lr = 2e-5
warmup_steps = 100

print("Training ...\n")
trainer.debug(epochs, lr, warmup_steps)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


Training ...

Epoch: 0
	Training Loss: 1.6067576762449516
	Validation Loss: 1.5043598476807463 

Epoch: 1
	Training Loss: 1.482056653726709
	Validation Loss: 1.5041712316842657 

